# 1. Creating pseudobulk expression values

Here, we will show how to generate your own pseudobulk values. To do so, you need a single-cell dataset and annotations. Here, we will use the FACS-sorted datasets from the TM. These datasets can be downloaded [here](https://figshare.com/projects/Tabula_Muris_Transcriptomic_characterization_of_20_organs_and_tissues_from_Mus_musculus_at_single_cell_resolution/27733). 

This dataset is not an AnnData object yet, so we will create that first. If you have an AnnData object already, you can skip these steps.

In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix as csr
from scipy.stats import pearsonr
import seaborn as sns
import h5py

### Load relevant FACS datasets

Limb muscle, lung, gland, spleen and marrow.

In [2]:
def read_facs(fn, anno_facs):
    '''
    fn = filename of the csv file
    '''
    x = sc.read_csv(fn)
    x = x.transpose()
    gn = x.var_names
    
    #Get annotations
    x.obs['celltype'] = anno_facs['cell_ontology_class']
    
    return x, gn    

In [3]:
# Read annotations
# Remove cells without cell ontology class
anno_facs_all = pd.read_csv('../../TM data/FACS/annotations_facs.csv', usecols=[2,3,21], index_col=0)

idx_remove = anno_facs_all['cell_ontology_class'].isna()
anno_facs_all = anno_facs_all[idx_remove==False]
anno_facs_all.iloc[:5]

,cell_ontology_class,tissue
cell,,
A1.B000610.3_56_F.1.1,bladder cell,Bladder
A1.B002764.3_38_F.1.1,bladder urothelial cell,Bladder
A1.B002771.3_39_F.1.1,bladder cell,Bladder
A1.D041914.3_8_M.1.1,bladder cell,Bladder
A1.D042253.3_9_M.1.1,bladder cell,Bladder


In [4]:
# Create anndata objects for every tissue 
facs_muscle, gn_muscle = read_facs('../../TM data/FACS/FACS/Limb_Muscle-counts.csv', anno_facs_all)
facs_lung, gn_lung = read_facs('../../TM data/FACS/FACS/Lung-counts.csv', anno_facs_all)
facs_gland, gn_gland = read_facs('../../TM data/FACS/FACS/Mammary_Gland-counts.csv', anno_facs_all)
facs_marrow, gn_marrow = read_facs('../../TM data/FACS/FACS/Marrow-counts.csv', anno_facs_all)
facs_spleen, gn_spleen = read_facs('../../TM data/FACS/FACS/Spleen-counts.csv', anno_facs_all)

#### Ensure same gene order

This makes it easier to compare the results from the cross-validation later on (then we're sure that the same genes were in the train/val/test set)

In [5]:
# Read gene names with TSS
hf = h5py.File('../../Zenodo/mouse/mouse_seq_hl.h5', 'r', libver='latest', swmr=True)
gn = np.asarray(hf['geneName']).astype('U30')
hf.close()

In [6]:
facs_gland = facs_gland[:, gn]
facs_muscle = facs_muscle[:, gn]
facs_marrow = facs_marrow[:, gn]
facs_lung = facs_lung[:, gn]
facs_spleen = facs_spleen[:, gn]

### Creating pseudobulk values

We do this tissue-specific (all cells) and cell population-specific (based on the annotations)

In [7]:
def create_labels_withoutpseudo(adata, tissue):
    
    ct = np.unique(np.array(adata.obs['celltype'], dtype=str))
    if np.isin('nan', ct):
        num_col = len(ct)
        ct = ct[ct != 'nan']
    else:
        num_col = len(ct)+1
    
    # Pseudobulk over whole tissue
    logmean=pd.DataFrame(data=np.zeros((np.shape(adata)[1], num_col)),
                         index=adata.var_names,
                         columns=np.concatenate((['All'], ct)))
    
    mn = np.asarray(np.mean(adata.X, axis=0)).reshape((-1,))
    idx = mn != 0
    mn[idx] = np.log10(mn[idx])
    mn[~idx] = -4 # Replace genes with 0 expression with -4
    logmean['All'] = mn
    
    # Pseudobulk per cell population
    for cp in np.unique(np.array(adata.obs['celltype'],dtype=str)):
        if cp != 'nan':

            idx = adata.obs['celltype'] == cp
            sub = adata[idx]

            mn = np.asarray(np.mean(sub.X, axis=0)).reshape((-1,))
            idx = mn != 0
            mn[idx] = np.log10(mn[idx])
            mn[~idx] = -4
            logmean[cp] = mn
            
    return logmean

In [8]:
x_facs_muscle = create_labels_withoutpseudo(facs_muscle, 'muscle_FACS')
x_facs_marrow = create_labels_withoutpseudo(facs_marrow, 'marrow_FACS')
x_facs_lung = create_labels_withoutpseudo(facs_lung, 'lung_FACS')
x_facs_gland = create_labels_withoutpseudo(facs_gland, 'gland_FACS')
x_facs_spleen = create_labels_withoutpseudo(facs_spleen, 'spleen_FACS')

In [9]:
# First column indicates the tissue-specific values
# Other columns indicate the cell populaiton-specific values
# You can save these values to a .csv file and use to train the model

x_facs_muscle

,All,B cell,T cell,endothelial cell,macrophage,mesenchymal stem cell,skeletal muscle satellite cell
0610007C21Rik,1.959240,1.815447,1.259389,1.853681,1.816020,2.310397,1.796099
0610007L01Rik,1.734659,1.394748,1.525600,1.421907,1.716930,1.710334,1.871692
0610007P08Rik,0.897297,0.906896,1.258021,0.981758,0.658541,1.000841,0.839548
0610007P14Rik,1.076247,1.136855,1.171935,0.905394,1.451616,1.331968,0.898339
0610007P22Rik,0.919383,1.046369,0.262112,0.501088,0.542687,0.799501,1.078913
...,...,...,...,...,...,...,...
Zyx,2.067279,1.239705,1.840823,1.757923,2.243974,2.146573,2.162946
Zzef1,1.188905,1.244956,0.796376,1.405996,1.538239,1.366097,0.981181
Zzz3,1.205109,0.852892,1.080214,1.280049,0.850578,1.311836,1.233043
a,-1.542561,-4.000000,-4.000000,-4.000000,-4.000000,-1.809560,-1.269996
